- Databricks Notebook: Lakehouse SQL Exercises
- Author: TheDataLead Databricks Workshop
- Description: Medallion architecture exercises using SQL (Bronze → Silver → Gold)

In [0]:
%sql
-- Run this to list the files in the S3 bucket
List 's3://thedatalead-data-engineering-projects-ingestion/workshop-demo/'

In [0]:
%sql
-- Run this to use catalog and schema
use catalog demo_catalog;
use library_schema

In [0]:
%sql
-- Bronze Layer: Ingest raw CSV files
-- TODO 1: Replace with actual CSV path if needed

-- Load books.csv
DROP TABLE IF EXISTS books_bronze;
CREATE books_bronze
USING CSV
OPTIONS (
  path = '--------',
  header = 'true',
  inferSchema = 'true'
);

-- TODO 2: Repeat the same steps to create borrowers and staff from their respective files
-- Hint: find the right path and replace in the path SQL create statement

-- Silver Layer: Clean and enrich the data
-- TODO 3: Clean books table (handle NULLs and cast publish_date)
-- Fill in the missing COALESCE defaults as needed
CREATE OR REPLACE TABLE books_silver AS
SELECT
  isbn,
  title,
  author,
  genre,
  CAST(publish_date AS DATE) AS publish_date,
  CAST(pages AS INT) AS pages
FROM books_bronze;

-- TODO 4: Clean borrowers table and compute return_delay_days
-- Hint: Use DATEDIFF between return_date and borrow_date
CREATE OR REPLACE TABLE borrowers_silver AS
SELECT
  user_id,
  name,
  book_isbn,
  CAST(borrow_date AS DATE) AS borrow_date,
  CAST(return_date AS DATE) AS return_date,
  -- Add return_delay_days calculation here
  NULL AS return_delay_days
FROM borrowers_bronze;

-- TODO 5: Clean staff table and cast hire_date
CREATE OR REPLACE TABLE staff_silver AS
SELECT
  staff_id,
  name,
  role,
  CAST(hire_date AS DATE) AS hire_date
FROM staff_bronze;

-- Gold Layer: Business Metrics
-- TODO 6: Most Borrowed Books
-- Count how many times each book is borrowed
CREATE OR REPLACE TABLE most_borrowed_books_gold AS
SELECT
  b.title,
  COUNT(*) AS borrow_count
FROM borrowers_silver br
JOIN books_silver b ON br.book_isbn = b.isbn
GROUP BY b.title
ORDER BY borrow_count DESC;

-- TODO 7: Average return delay by genre
CREATE OR REPLACE TABLE delay_by_genre_gold AS
SELECT
  b.genre,
  -- Calculate AVG delay here
  NULL AS avg_return_delay_days
FROM borrowers_silver br
JOIN books_silver b ON br.book_isbn = b.isbn
GROUP BY b.genre;

-- TODO 8: Count staff by role
CREATE OR REPLACE TABLE staff_count_by_role_gold AS
SELECT
  role,
  COUNT(*) AS staff_count
FROM staff_silver
GROUP BY role;

-- ✅ Bonus Challenge: Create gold.staff_hired_per_year
-- Hint: Use YEAR(hire_date) to group by year
